**Basic Configuration**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install CocoDataset==0.1.2


In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip /content/annotations_trainval2017.zip


--2022-04-28 01:48:16--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.205.195
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.205.195|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  46.9MB/s    in 5.6s    

2022-04-28 01:48:21 (43.4 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  /content/annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


**Original Json file -> Filtered Json File**

In [ ]:
import json
fp = open("/content/annotations/person_keypoints_train2017.json")
Keypoints_Json = json.load(fp)

In [ ]:
selected_annotations = []
selected_images = []
selected_img_id =[]
length_anno = len(Keypoints_Json["annotations"])
length_img = len(Keypoints_Json["images"])
print("num annotation:", length_anno)
print("num images:", length_img)
count = 0
for i in range(length_anno):
  if (Keypoints_Json["annotations"][i]["num_keypoints"] >= 16):
    count+=1
    selected_annotations.append(Keypoints_Json["annotations"][i])
    selected_img_id.append(Keypoints_Json["annotations"][i]["image_id"])
print("num annoations with keypoints greater than 8: ",count)
         
          


num annotation: 262465
num images: 118287
num annoations with keypoints greater than 8:  24123


In [ ]:
img_img_id = []
count = 0
for i in range(length_img):
  if(Keypoints_Json["images"][i]["id"] in selected_img_id):
    count += 1
    img_img_id.append(Keypoints_Json["images"][i]["id"])
    selected_images.append(Keypoints_Json["images"][i])
print("number of image id's that match", count)

number of image id's that match 16986


In [ ]:
revised_selected_annotations = []
count = 0
for i in range(len(selected_annotations)):
  if(selected_annotations[i]["image_id"] in img_img_id):
    revised_selected_annotations.append(selected_annotations[i])
    count += 1
print(count)

print(len(revised_selected_annotations))
print(len(selected_images))

116021
116021
49431


In [ ]:
Keypoints_Json_selected= Keypoints_Json
Keypoints_Json_selected["annotations"] = revised_selected_annotations
Keypoints_Json_selected["images"]= selected_images

In [ ]:
with open("/content/drive/MyDrive/HTL/person_annotations.json", "w") as outfile:
    json.dump(Keypoints_Json_selected, outfile)
with open("/content/drive/Shareddrives/CSCI5561/Project/TransferLearning/person_annotations.json", "w") as outfile:
    json.dump(Keypoints_Json_selected, outfile)

In [ ]:
import json

fp2 = open("/content/drive/MyDrive/HTL/person_annotations.json")
img_anotation_json = json.load(fp2)
print(len(img_anotation_json["annotations"]))
for i in range(len(img_anotation_json["annotations"])):
  temp = []
  for j in range(0,len(img_anotation_json["annotations"][i]["keypoints"]) - 1):
    if ((j+1) % 3 == 0 and j > 0):
      continue
    else:
      temp.append(img_anotation_json["annotations"][i]["keypoints"][j])
  img_anotation_json["annotations"][i]["keypoints"] = temp
print(img_anotation_json["annotations"][0]["keypoints"])
with open("/content/drive/MyDrive/HTL/person_annotations_no_vis.json", "w") as outfile:
    json.dump(img_anotation_json, outfile)

116021
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 325, 160, 398, 177, 0, 0, 437, 238, 0, 0, 477, 270, 287, 255, 339, 267, 0, 0, 423, 314, 0, 0, 355, 367]




"human keypoints": [ 0-1"nose", 2-3"left_eye", 4-5"right_eye", 6-7"left_ear", 8-9"right_ear", 10-11"left_shoulder", 12-13"right_shoulder", 14-15"left_elbow", 16-17"right_elbow", 18-19"left_wrist", 20-21"right_wrist", 22-23"left_hip", 24-25"right_hip", 26-27"left_knee", 28-29"right_knee", 30-31"left_ankle", 32-33"right_ankle" ]

monkey keypoints 0-1:Right eye, 2-3:left eye, 4-5:Nose, 6-7:Head, 8-9:Neck, 10-11:Right shoulder, 12-13: Right elbow, 14-15:right wrist, 16-17:Left shoulder, 18-19:Left elbow, 20-21:Left wrist, 22-23:Hip, 24-25: Right knee, 26-27: Right ankle, 28-29: Left knee, 30-31:left ankle, 32-33:tail

In [ ]:
import json
fp = open("/content/drive/Shareddrives/CSCI5561/Project/OpenMonkeyChallenge/Annotations/train_annotation.json")
fp2 = open("/content/drive/MyDrive/HTL/person_annotations_no_vis.json")

monkey_Keypoints_Json = json.load(fp)
human_Keypoints_Json = json.load(fp2)

print("Length of monkey json: ",len(monkey_Keypoints_Json["data"]))
print("Length of human json: ",len(human_Keypoints_Json["images"]))

for i in range(len(human_Keypoints_Json["images"])):
  monkey_Keypoints_Json["data"][i]["file"] = human_Keypoints_Json["images"][i]["file_name"]
  monkey_Keypoints_Json["data"][i]["species"] = "Human"
  monkey_Keypoints_Json["data"][i]["bbox"] = int(human_Keypoints_Json["annotations"][i]["bbox"])
  #map landmarks and visibility properly
  monkey_Keypoints_Json["data"][i]["landmarks"][0] = human_Keypoints_Json["annotations"][i]["keypoints"][4]
  monkey_Keypoints_Json["data"][i]["landmarks"][1] = human_Keypoints_Json["annotations"][i]["keypoints"][5]
  monkey_Keypoints_Json["data"][i]["landmarks"][2] = human_Keypoints_Json["annotations"][i]["keypoints"][2]
  monkey_Keypoints_Json["data"][i]["landmarks"][3] = human_Keypoints_Json["annotations"][i]["keypoints"][3]
  monkey_Keypoints_Json["data"][i]["landmarks"][4] = human_Keypoints_Json["annotations"][i]["keypoints"][0]
  monkey_Keypoints_Json["data"][i]["landmarks"][5] = human_Keypoints_Json["annotations"][i]["keypoints"][1]
  monkey_Keypoints_Json["data"][i]["landmarks"][6] = 0
  monkey_Keypoints_Json["data"][i]["landmarks"][7] = 0
  monkey_Keypoints_Json["data"][i]["landmarks"][8] = 0
  monkey_Keypoints_Json["data"][i]["landmarks"][9] = 0
  monkey_Keypoints_Json["data"][i]["landmarks"][10] = human_Keypoints_Json["annotations"][i]["keypoints"][12]
  monkey_Keypoints_Json["data"][i]["landmarks"][11] = human_Keypoints_Json["annotations"][i]["keypoints"][13]
  monkey_Keypoints_Json["data"][i]["landmarks"][12] = human_Keypoints_Json["annotations"][i]["keypoints"][16]
  monkey_Keypoints_Json["data"][i]["landmarks"][13] = human_Keypoints_Json["annotations"][i]["keypoints"][17]
  monkey_Keypoints_Json["data"][i]["landmarks"][14] = human_Keypoints_Json["annotations"][i]["keypoints"][20]
  monkey_Keypoints_Json["data"][i]["landmarks"][15] = human_Keypoints_Json["annotations"][i]["keypoints"][21]
  monkey_Keypoints_Json["data"][i]["landmarks"][16] = human_Keypoints_Json["annotations"][i]["keypoints"][10]
  monkey_Keypoints_Json["data"][i]["landmarks"][17] = human_Keypoints_Json["annotations"][i]["keypoints"][11]
  monkey_Keypoints_Json["data"][i]["landmarks"][18] = human_Keypoints_Json["annotations"][i]["keypoints"][14]
  monkey_Keypoints_Json["data"][i]["landmarks"][19] = human_Keypoints_Json["annotations"][i]["keypoints"][15]
  monkey_Keypoints_Json["data"][i]["landmarks"][20] = human_Keypoints_Json["annotations"][i]["keypoints"][18]
  monkey_Keypoints_Json["data"][i]["landmarks"][21] = human_Keypoints_Json["annotations"][i]["keypoints"][19]
  monkey_Keypoints_Json["data"][i]["landmarks"][22] = int((human_Keypoints_Json["annotations"][i]["keypoints"][22] + human_Keypoints_Json["annotations"][i]["keypoints"][24]) /2)
  monkey_Keypoints_Json["data"][i]["landmarks"][23] = int((human_Keypoints_Json["annotations"][i]["keypoints"][23] + human_Keypoints_Json["annotations"][i]["keypoints"][25]) /2)
  monkey_Keypoints_Json["data"][i]["landmarks"][24] = human_Keypoints_Json["annotations"][i]["keypoints"][28]
  monkey_Keypoints_Json["data"][i]["landmarks"][25] = human_Keypoints_Json["annotations"][i]["keypoints"][29]
  monkey_Keypoints_Json["data"][i]["landmarks"][26] = human_Keypoints_Json["annotations"][i]["keypoints"][32]
  monkey_Keypoints_Json["data"][i]["landmarks"][27] = human_Keypoints_Json["annotations"][i]["keypoints"][33]
  monkey_Keypoints_Json["data"][i]["landmarks"][28] = human_Keypoints_Json["annotations"][i]["keypoints"][26]
  monkey_Keypoints_Json["data"][i]["landmarks"][29] = human_Keypoints_Json["annotations"][i]["keypoints"][27]
  monkey_Keypoints_Json["data"][i]["landmarks"][30] = human_Keypoints_Json["annotations"][i]["keypoints"][30]
  monkey_Keypoints_Json["data"][i]["landmarks"][31] = human_Keypoints_Json["annotations"][i]["keypoints"][31]
  monkey_Keypoints_Json["data"][i]["landmarks"][32] = 0
  monkey_Keypoints_Json["data"][i]["landmarks"][33] = 0


del monkey_Keypoints_Json["data"][49431:66917]
print("Length of monkey json: ",len(monkey_Keypoints_Json["data"]))
print("Length of human json: ",len(human_Keypoints_Json["images"]))

with open("/content/drive/MyDrive/HTL/person_annotations_OMC_format.json", "w") as outfile:
    json.dump(monkey_Keypoints_Json, outfile)
with open("/content/drive/Shareddrives/CSCI5561/Project/TransferLearning/person_annotations_OMC_format.json", "w") as outfile:
    json.dump(monkey_Keypoints_Json, outfile)

Length of monkey json:  66917
Length of human json:  49431
Length of monkey json:  49431
Length of human json:  49431


**Json File output to the .json**

In [ ]:
from pycocotools.coco import COCO
import requests
import os

coco = COCO("/content/drive/MyDrive/HTL/person_annotations.json")
# Specify a list of category names of interest
catIds = coco.getCatIds(catNms=["person"])
# Get the corresponding image ids and images using loadImgs
imgIds = coco.getImgIds(catIds=catIds)
images = coco.loadImgs(imgIds)
# Save the images into a local folder
count=0
# specified count images for class name
for im in images:
    img_data = requests.get(im['coco_url']).content
    with open("/content/drive/Shareddrives/CSCI5561/Project/TransferLearning/images" +'/'+ im['file_name'], 'wb') as handler:
        handler.write(img_data)
    count+=1
    if count>200:
        print('finished images download')
        break
    print('no.of image:',count)

In [ ]:
from coco_dataset import coco_dataset_download as cocod

class_name='person'  #class name example 
images_count= 50000     #count of images  
annotations_path='/content/drive/MyDrive/HTL/person_annotations.json' #path of coco dataset annotations 
#call download function 
cocod.coco_dataset_download(class_name,images_count,annotations_path)




loading annotations into memory...
Done (t=8.60s)
creating index...
index created!


FileExistsError: ignored

In [ ]:
# loop over json object
# for every single file name, to see if it exists in the file we downloaded earlier
from os.path import exists
fp2 = open("/content/drive/MyDrive/cv_proj/person_keypoints_selectedTrain.json")
revision_keypoints_json = json.load(fp2)
tes_count = 0
for i in range (len(revision_keypoints_json["images"])):
  path_to_file ="/content/drive/MyDrive/cv_proj/person/"+ revision_keypoints_json["images"][i]["file_name"]
  if(exists(path_to_file)):
    tes_count +=1
print(tes_count)


4847


**Revision of Keypoints**

In [ ]:
print(revision_keypoints_json["annotations"][0]["keypoints"])
print(revision_keypoints_json["annotations"][0]["image_id"])
for i in range(len(revision_keypoints_json["images"])):
  if revision_keypoints_json["images"][i]["id"]==44474 :
    print(revision_keypoints_json["images"][i]["file_name"])
    break


[145, 113, 2, 158, 96, 2, 124, 97, 2, 184, 85, 2, 84, 89, 2, 184, 141, 2, 77, 153, 2, 219, 205, 2, 67, 229, 2, 183, 193, 2, 125, 182, 2, 174, 279, 2, 97, 287, 2, 223, 230, 2, 50, 302, 2, 287, 312, 2, 70, 372, 2]
44474
COCO_train2014_000000044474.jpg


Deleting the ears and Merge the hips

In [ ]:
fp2 = open("/content/drive/MyDrive/cv_proj/person_keypoints_selectedTrain.json")
img_anotation_json = json.load(fp2)
print("before deleteting ears and merge hip ",len(img_anotation_json["annotations"][0]["keypoints"]))
for i in range(len(img_anotation_json["annotations"])):
  anno_index = img_anotation_json["annotations"][i]["keypoints"]
  # 12 and 13 keypoints on hip 33 34 35 36 37 38 
  anno_index[33]= int((anno_index[33] +anno_index[36])/2)
  anno_index[34]= int((anno_index[34] +anno_index[37])/2)
  anno_index[35]= int((anno_index[35] +anno_index[38])/2)
  for j in range(12):
    anno_index[j+36]=anno_index[j+39]
  del anno_index[48:52]
  #delete the ears
  del anno_index[10:16]
  img_anotation_json["annotations"][i]["keypoints"] = anno_index
print("after deleteting ears and merge hip",len(anno_index))
with open("/content/drive/MyDrive/cv_proj/img_anot.json", "w") as outfile:
    json.dump(img_anotation_json, outfile)

before deleteting ears and merge hip  51
after deleteting ears and merge hip 42


In [ ]:
import cv2
import os
import random
import json
import shutil

# Keypoint visualization(teal color centered at keypoint)

fp3 = open("/content/drive/MyDrive/HTL/person_annotations.json")
img_anotation_json = json.load(fp3)

output_dir = "/content/drive/MyDrive/HTL/image_visualization"
if os.path.isdir(output_dir):
  shutil.rmtree(output_dir) 

for i in range(5):
  idx = random.randint(0, 4846)
  img_path ="/content/drive/Shareddrives/CSCI5561/Project/TransferLearning/images"+ img_anotation_json["images"][idx]["file_name"]
  img = cv2.imread(img_path)

  for j in range ((len(img_anotation_json["annotations"]))):
    if img_anotation_json["annotations"][j]["image_id"]== img_anotation_json["images"][idx]["id"]:
      keypoints_anno = j
      break
  keypoints = img_anotation_json["annotations"][keypoints_anno]["keypoints"]
      
  k = 0
  while(k<len(keypoints)):
      x = keypoints[k]
      y = keypoints[k+1]
      k+=3
      for v in range(3):
        for u in range(3):
          img[y+v][x+u] = [255, 0, 0]

  if not os.path.exists(output_dir):
    print("MADE DIRECTORY: " + output_dir)
    os.makedirs(output_dir)

  cv2.imwrite(output_dir + img_anotation_json["images"][idx]["file_name"], img)

TypeError: ignored